In [40]:
# Import librerias
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Se inicia la Limpieza de los datasets de Evaluación Nacional Aprender


* Licencia: CC-BY-2.5-AR
* Frecuencia de actualización: Anualmente
* Responsable: Secretaría de Educación 
* Fecha de publicación: 2016
* Fecha de actualización: 2024
* Fuente primaria: Datos abiertos de la Secretaría de Educación 


## Se inicia la exploracion de los datasets de Evaluación Nacional Aprender


* Licencia: CC-BY-2.5-AR
* Frecuencia de actualización: Anualmente
* Responsable: Secretaría de Educación 
* Fecha de publicación: 2016
* Fecha de actualización: 2024
* Fuente primaria: Datos abiertos de la Secretaría de Educación 


In [12]:
#markdown de que tratan las columnas
#hacer un grafico de una pregunta interesante (grupo de columnas) 
#primeras 15 patrick
#siguient bruno 
#34 a final juan
#https://docs.google.com/document/d/1xeXGwWcrq0DfJiaZdMEY90f4sume7BjmDSZ1k1nbNno/edit?usp=sharing
#https://data.educacion.gob.ar/aprender-respuestas-estudiante

In [41]:
ruta_archivo_leng = "../data/raw/2024 Base APRENDER - Censal - Secundaria 5-6 año - Agregada - Desempeños de Lengua(in).csv"
ruta_archivo_mate = "../data/raw/2024 Base APRENDER - Censal - Secundaria 5-6 año - Agregada - Desempeños de Matematica(in).csv"

ruta_archivo_leng_pre = "../data/interim/2024_Ap_56Lengcoma.csv"
ruta_archivo_mate_pre = "../data/interim/2024_Ap_56Matecoma.csv"

ruta_archivo_leng_pre2 = "../data/interim/2024_Ap_56Lengceros.csv"
ruta_archivo_mate_pre2 = "../data/interim/2024_Ap_56Mateceros.csv"

ruta_archivo_final = "../data/processed/2024_Ap_56listo.csv"



* Debido a que el dataset tiene una codificacion latin-1 y
  utiliza el coma como punto decimal, esta funcion reemplaza por punto las comas

In [42]:
def reemplazar_comas_por_puntos(input_csv, output_csv):
    """
    Reemplaza todas las comas por puntos en un archivo CSV con separador ';'
    y guarda el resultado en un nuevo archivo.
    
    :param input_csv: Ruta del archivo CSV original
    :param output_csv: Ruta del archivo CSV de salida
    """
    with open(input_csv, 'r', encoding='latin-1') as f_in:
        contenido = f_in.read()
    
    # Reemplaza todas las comas por puntos
    contenido = contenido.replace(',', '.')
    
    with open(output_csv, 'w', encoding='utf-8') as f_out:
        f_out.write(contenido)

# Ejemplo de uso
reemplazar_comas_por_puntos(ruta_archivo_leng, ruta_archivo_leng_pre)
reemplazar_comas_por_puntos(ruta_archivo_mate, ruta_archivo_mate_pre)

* Se abre el nuevo archivo generado y se comprueba su estado

In [43]:
df1 = pd.read_csv(ruta_archivo_leng_pre, sep=';', decimal=".", low_memory=False)
df2 = pd.read_csv(ruta_archivo_mate_pre, sep=';', decimal=".", low_memory=False)

In [44]:
df1.head(2)

,jurisdiccion,departamento,sector,ambito,ap01_Blanco,ap01_No_disponible,ap01_Multimarca,ap01_Enero,ap01_Febrero,ap01_Marzo,...,Nivel_Ed_Persona_Resp_Blanco,Nivel_Ed_Persona_Resp_No_disponible,Nivel_Ed_Persona_Resp_Multimarca,Nivel_Ed_Persona_Resp_No_fue_a_la_escuela,Nivel_Ed_Persona_Resp_Primaria_incompleto,Nivel_Ed_Persona_Resp_Primaria_completo,Nivel_Ed_Persona_Resp_Secundaria_incompleto,Nivel_Ed_Persona_Resp_Secundaria_completo,Nivel_Ed_Persona_Resp_Terciario_universitario_posgrado_incomple,Nivel_Ed_Persona_Resp_Terciario_universitario_posgrado_completo
0,Ciudad Autónoma de Buenos Aires,Comuna 1,Estatal,Urbano,,,,162.0171204,168.4406553,179.9559899,...,,,,11.9878283,18.4604794,18.5340397,49.8504244,85.80181,29.0259314,156.0451127
1,Ciudad Autónoma de Buenos Aires,Comuna 1,Privado,Urbano,,,,152.8266478,114.963918,112.7640288,...,,,,5.9561216,2.3819654,8.4506522,15.9379779,39.5181871,28.081318,110.2102714


* Como se observa que hay columnas vacias, se prodece a procesarlas para ponerlas 
 con el valor 0 de tipo float, dejando sin cambios las primeras 4 columnas que son categoricas

In [100]:
df1.columns

Index(['jurisdiccion', 'departamento', 'sector', 'ambito', 'ap01_Blanco',
       'ap01_No_disponible', 'ap01_Multimarca', 'ap01_Enero', 'ap01_Febrero',
       'ap01_Marzo',
       ...
       'Nivel_Ed_Persona_Resp_Blanco', 'Nivel_Ed_Persona_Resp_No_disponible',
       'Nivel_Ed_Persona_Resp_Multimarca',
       'Nivel_Ed_Persona_Resp_No_fue_a_la_escuela',
       'Nivel_Ed_Persona_Resp_Primaria_incompleto',
       'Nivel_Ed_Persona_Resp_Primaria_completo',
       'Nivel_Ed_Persona_Resp_Secundaria_incompleto',
       'Nivel_Ed_Persona_Resp_Secundaria_completo',
       'Nivel_Ed_Persona_Resp_Terciario_universitario_posgrado_incomple',
       'Nivel_Ed_Persona_Resp_Terciario_universitario_posgrado_completo'],
      dtype='object', length=1035)

In [45]:
def rellenar_y_convertir_columnas_desde_4(df: pd.DataFrame) -> pd.DataFrame:
    """
    Rellena los valores vacíos o con solo espacios con 0 y convierte a float
    todas las columnas desde la 4ta hasta la última del DataFrame.
    Evita el warning de downcasting de pandas.
    """
    columnas = df.columns[4:]
    
    for col in columnas:
        # Convertimos a string primero para que replace no genere warning
   #     df[col] = df[col].astype(str).replace(r'^\s*$', '0', regex=True)
        # Convertimos a float
  #      df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(float)
        df[col] = df[col].mask(df[col].astype(str).str.strip() == '', 0)
        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(float)
    
    return df

# Ejemplo de uso
#df = pd.read_csv("archivo.csv")
df1 = rellenar_y_convertir_columnas_desde_4(df1)
df2 = rellenar_y_convertir_columnas_desde_4(df2)

In [46]:
df1.head()

,jurisdiccion,departamento,sector,ambito,ap01_Blanco,ap01_No_disponible,ap01_Multimarca,ap01_Enero,ap01_Febrero,ap01_Marzo,...,Nivel_Ed_Persona_Resp_Blanco,Nivel_Ed_Persona_Resp_No_disponible,Nivel_Ed_Persona_Resp_Multimarca,Nivel_Ed_Persona_Resp_No_fue_a_la_escuela,Nivel_Ed_Persona_Resp_Primaria_incompleto,Nivel_Ed_Persona_Resp_Primaria_completo,Nivel_Ed_Persona_Resp_Secundaria_incompleto,Nivel_Ed_Persona_Resp_Secundaria_completo,Nivel_Ed_Persona_Resp_Terciario_universitario_posgrado_incomple,Nivel_Ed_Persona_Resp_Terciario_universitario_posgrado_completo
0,Ciudad Autónoma de Buenos Aires,Comuna 1,Estatal,Urbano,0.0,0.0,0.0,162.017120,168.440655,179.955990,...,0.0,0.0,0.0,11.987828,18.460479,18.534040,49.850424,85.801810,29.025931,156.045113
1,Ciudad Autónoma de Buenos Aires,Comuna 1,Privado,Urbano,0.0,0.0,0.0,152.826648,114.963918,112.764029,...,0.0,0.0,0.0,5.956122,2.381965,8.450652,15.937978,39.518187,28.081318,110.210271
2,Ciudad Autónoma de Buenos Aires,Comuna 10,Estatal,Urbano,0.0,0.0,0.0,76.834254,102.428661,91.227815,...,0.0,0.0,0.0,4.399404,6.581900,4.550060,32.354435,58.277813,18.116752,47.907880
3,Ciudad Autónoma de Buenos Aires,Comuna 10,Privado,Urbano,0.0,0.0,0.0,114.127398,84.716258,96.843710,...,0.0,0.0,0.0,5.844474,1.305030,2.507037,22.487811,42.116628,30.763302,105.837412
4,Ciudad Autónoma de Buenos Aires,Comuna 11,Estatal,Urbano,0.0,0.0,0.0,80.486393,57.005538,68.978603,...,0.0,0.0,0.0,3.070107,9.086502,1.482637,37.686962,53.557962,12.758785,30.186418


In [124]:
df2.head()

,jurisdiccion,departamento,sector,ambito,ap01_Blanco,ap01_No_disponible,ap01_Multimarca,ap01_Enero,ap01_Febrero,ap01_Marzo,...,Nivel_Ed_Persona_Resp_Blanco,Nivel_Ed_Persona_Resp_No_disponible,Nivel_Ed_Persona_Resp_Multimarca,Nivel_Ed_Persona_Resp_No_fue_a_la_escuela,Nivel_Ed_Persona_Resp_Primaria_incompleto,Nivel_Ed_Persona_Resp_Primaria_completo,Nivel_Ed_Persona_Resp_Secundaria_incompleto,Nivel_Ed_Persona_Resp_Secundaria_completo,Nivel_Ed_Persona_Resp_Terciario_universitario_posgrado_incomple,Nivel_Ed_Persona_Resp_Terciario_universitario_posgrado_completo
0,Ciudad Autónoma de Buenos Aires,Comuna 1,Estatal,Urbano,0.0,0.0,0.0,164.371787,176.216148,178.096707,...,0.0,0.0,0.0,11.969616,18.600477,21.281081,39.496587,83.687215,24.519076,158.402045
1,Ciudad Autónoma de Buenos Aires,Comuna 1,Privado,Urbano,0.0,0.0,0.0,153.026797,114.800153,113.129972,...,0.0,0.0,0.0,5.952883,0.000000,8.288188,15.941491,39.708244,27.785983,110.507941
2,Ciudad Autónoma de Buenos Aires,Comuna 10,Estatal,Urbano,0.0,0.0,0.0,76.240297,105.053777,88.027701,...,0.0,0.0,0.0,3.050054,6.750184,5.037498,33.138638,58.556008,18.777238,48.785247
3,Ciudad Autónoma de Buenos Aires,Comuna 10,Privado,Urbano,0.0,0.0,0.0,113.496421,85.540429,97.949533,...,0.0,0.0,0.0,5.713592,1.304320,2.550951,21.050045,44.947701,31.876019,104.403566
4,Ciudad Autónoma de Buenos Aires,Comuna 11,Estatal,Urbano,0.0,0.0,0.0,81.107824,57.527755,65.355275,...,0.0,0.0,0.0,2.787548,4.512754,1.410500,39.858709,49.585557,12.983318,25.361202


## Con ambos df procesados

* se procede a guardarlos

In [47]:
def guardar_df(df: pd.DataFrame, ruta: str):
    """
    Guarda un DataFrame en un archivo CSV en la ruta indicada.

    Parámetros:
    df (pd.DataFrame): DataFrame a guardar.
    ruta (str): Ruta completa del archivo donde se guardará el CSV.
    """
    try:
        df.to_csv(ruta, index=False, encoding='utf-8')
        print(f"Archivo guardado exitosamente en: {ruta}")
    except Exception as e:
        print(f"Error al guardar el archivo: {e}")

guardar_df(df1, ruta_archivo_leng_pre2)
guardar_df(df2, ruta_archivo_mate_pre2)


Archivo guardado exitosamente en: ../data/interim/2024_Ap_56Lengceros.csv
Archivo guardado exitosamente en: ../data/interim/2024_Ap_56Mateceros.csv


In [54]:
df1 = pd.read_csv(ruta_archivo_leng_pre2, low_memory=False)
df2 = pd.read_csv(ruta_archivo_mate_pre2, low_memory=False)

In [55]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1228 entries, 0 to 1227
Columns: 1035 entries, jurisdiccion to Nivel_Ed_Persona_Resp_Terciario_universitario_posgrado_completo
dtypes: float64(1031), object(4)
memory usage: 9.7+ MB


In [59]:
df1.dtypes

jurisdiccion                                                        object
departamento                                                        object
sector                                                              object
ambito                                                              object
ap01_Blanco                                                        float64
ap01_No_disponible                                                 float64
ap01_Multimarca                                                    float64
ap01_Enero                                                         float64
ap01_Febrero                                                       float64
ap01_Marzo                                                         float64
ap01_Abril                                                         float64
ap01_Mayo                                                          float64
ap01_Junio                                                         float64
ap01_Julio               

## Se procede a comparar ambos dataset el de lengua y matematica

In [60]:
#import pandas as pd

# Cargar ambos CSV
#df1 = pd.read_csv(ruta_archivo_leng, sep=';', decimal=",", encoding="latin-1")
#df2 = pd.read_csv(ruta_archivo_mate, sep=';', decimal=",",  encoding="latin-1")



# Comparar dimensiones (deberían ser iguales)
print("Shape archivo1:", df1.shape)
print("Shape archivo2:", df2.shape)




Shape archivo1: (1228, 1035)
Shape archivo2: (1228, 1035)


In [61]:
print(df1.columns)

Index(['jurisdiccion', 'departamento', 'sector', 'ambito', 'ap01_Blanco',
       'ap01_No_disponible', 'ap01_Multimarca', 'ap01_Enero', 'ap01_Febrero',
       'ap01_Marzo',
       ...
       'Nivel_Ed_Persona_Resp_Blanco', 'Nivel_Ed_Persona_Resp_No_disponible',
       'Nivel_Ed_Persona_Resp_Multimarca',
       'Nivel_Ed_Persona_Resp_No_fue_a_la_escuela',
       'Nivel_Ed_Persona_Resp_Primaria_incompleto',
       'Nivel_Ed_Persona_Resp_Primaria_completo',
       'Nivel_Ed_Persona_Resp_Secundaria_incompleto',
       'Nivel_Ed_Persona_Resp_Secundaria_completo',
       'Nivel_Ed_Persona_Resp_Terciario_universitario_posgrado_incomple',
       'Nivel_Ed_Persona_Resp_Terciario_universitario_posgrado_completo'],
      dtype='object', length=1035)


In [62]:
# Listar las columnas de cada uno
cols1 = set(df1.columns)
cols2 = set(df2.columns)

#print("Columnas en archivo1:", cols1)
#print("Columnas en archivo2:", cols2)

# Columnas que comparten
comunes = cols1.intersection(cols2)
#print("\n✅ Columnas comunes:", comunes)

# Columnas que están en uno pero no en el otro
solo_en_1 = cols1 - cols2
solo_en_2 = cols2 - cols1

print("\n📌 Solo en archivo1:", solo_en_1)
print("📌 Solo en archivo2:", solo_en_2)


📌 Solo en archivo1: {'ldesemp_Satisfactorio', 'ldesemp_Por_debajo_del_nivel_básico', 'ldesemp_Básico', 'ldesemp_Avanzado'}
📌 Solo en archivo2: {'mdesemp_Por_debajo_del_nivel_básico', 'mdesemp_Satisfactorio', 'mdesemp_Básico', 'mdesemp_Avanzado'}


# Diferencias encontradas

hay columnas que estan en uno de ambos datasets, por lo que le vamos a hacer un join


Esto quiere decir que esas columnas las vamos a unir con un join,
pero vamos a ver la diferencia entre las otras

Pero antes, vamos a ver si difieren en ambos datos



In [63]:


# Columnas comunes (convertir a lista ordenada)
comunes = sorted(set(df1.columns).intersection(set(df2.columns)))

#print("Columnas comunes:", comunes)

# Filtrar ambos dataframes solo con las columnas comunes
df1 = df1[comunes]
df2 = df2[comunes]

#df1 = df1.reset_index(drop=True)
#df2 = df2.reset_index(drop=True)
# Ver en qué filas/columnas difieren
diff = (df1 != df2)

# Contar cuántas celdas son distintas
total_diferencias = diff.sum().sum()
#print("Total de diferencias:", total_diferencias)

# Mostrar las filas con diferencias
diferencias = df1[diff.any(axis=1)]
#print("Filas con diferencias:")
#print(diferencias)

# Si querés ver lado a lado:
comparacion = df1.compare(df2)
print("Comparacion")
print(comparacion.head(3))


Comparacion
  NSE_Quintiles_Q1             NSE_Quintiles_Q2             NSE_Quintiles_Q3  \
              self       other             self       other             self   
0       305.454077  311.912866       395.816310  397.506189       448.543402   
1        90.850428   89.540904       141.249120  139.963330       248.742979   
2       155.514010  149.735497       163.553295  158.893421       241.305655   

              NSE_Quintiles_Q4             NSE_Quintiles_Q5              ...  \
        other             self       other             self       other  ...   
0  434.697672       406.694880  412.287567       447.722680  444.684165  ...   
1  247.529080       343.896081  350.372409       515.733006  512.337019  ...   
2  249.487679       262.597119  267.873300       214.482164  215.257795  ...   

  sobreedad_1_año_de_sobreedad_18_años_al_30_de_junio              \
                                                 self       other   
0                                         354.27

# Vista en profundidad

debido a que ambos dataset tinen datos censurados, vamos a ver que tanto difieren

la solucion sera hacer un promedio de las columnas que son iguales


In [64]:
# Supongamos que tus DataFrames se llaman df1 y df2

# Columnas comunes
columnas_comunes = df1.columns.intersection(df2.columns)

# Excluir columnas de tipo object (strings)
columnas_numericas = [col for col in columnas_comunes if pd.api.types.is_numeric_dtype(df1[col])]

# Seleccionamos solo columnas numéricas y hacemos copia
df1_comun = df1[columnas_numericas].copy()
df2_comun = df2[columnas_numericas].copy()

# Convertir números con coma decimal a float
for col in columnas_numericas:
    df1_comun[col] = df1_comun[col].astype(str).str.replace(',', '.').astype(float)
    df2_comun[col] = df2_comun[col].astype(str).str.replace(',', '.').astype(float)

# Restar DataFrames
diferencias = df1_comun - df2_comun

# Estadísticas
pd.set_option('display.max_rows', None)
print(diferencias.describe())
print("Máxima diferencia por columna:\n", diferencias.max())
#print("Máxima diferencia absoluta total:", diferencias.abs().max().max())

       NSE_Quintiles_Q1  NSE_Quintiles_Q2  NSE_Quintiles_Q3  NSE_Quintiles_Q4  \
count       1228.000000       1228.000000       1228.000000       1228.000000   
mean           0.157314         -0.053904          0.105808         -0.066229   
std            3.618871          2.867323          2.993732          2.408772   
min          -29.805805        -37.746266        -26.559587        -17.428621   
25%           -0.486343         -0.610869         -0.464470         -0.447804   
50%            0.000000          0.000000          0.000000          0.000000   
75%            0.527975          0.569087          0.700515          0.365617   
max           59.526214         23.082405         29.589872         11.669179   

       NSE_Quintiles_Q5  Nivel_Ed_Madre_Blanco  Nivel_Ed_Madre_Multimarca  \
count       1228.000000                 1228.0                     1228.0   
mean          -0.142990                    0.0                        0.0   
std            3.036386                

In [65]:

recuento = ["ap01_Abril","ap01_Agosto","ap01_Blanco","ap01_Diciembre",
            "ap01_Enero","ap01_Febrero","ap01_Julio","ap01_Junio",
            "ap01_Marzo","ap01_Mayo","ap01_Multimarca", "ap01_No_disponible",
            "ap01_Noviembre","ap01_Octubre","ap01_Septiembre"]



df1['Nro_Encuestados'] = df1[recuento].sum(axis=1)
df2['Nro_Encuestados'] = df2[recuento].sum(axis=1)

In [66]:
print(df1['Nro_Encuestados'])
print(df2['Nro_Encuestados'])

0        2004.231349
1        1340.471615
2        1037.452244
3        1318.705478
4         814.644791
5        1519.396463
6         773.810754
7        1584.342555
8        1144.132923
9        2441.125303
10        829.452905
11       1208.246190
12        676.406191
13        805.436521
14       1065.247061
15        952.804121
16       1441.906252
17       1187.169311
18       1523.046105
19        977.896625
20        906.091503
21       1135.511170
22       1415.570890
23       1475.976664
24        998.172335
25       1048.968333
26       1557.105042
27       1183.825173
28       1113.729940
29       1476.124705
30        100.285788
31        313.549478
32         83.854922
33        129.476224
34        474.658797
35        155.143948
36         42.810686
37        169.305048
38         66.924840
39         33.413219
40         50.388407
41         21.927426
42         83.644756
43       5670.768510
44       2976.851049
45         34.457383
46        264.035256
47         52

#Esto se hace una sola vez

In [67]:
# Excluir columnas de tipo object (strings)
#columnas_numericas = [col for col in columnas_comunes if pd.api.types.is_numeric_dtype(df1[col])]


columnas_numericas = df1.columns.drop(["jurisdiccion","departamento","sector","ambito","Nro_Encuestados"]).tolist()
df1[columnas_numericas] = df1[columnas_numericas].div(df1["Nro_Encuestados"], axis=0)

columnas_numericas = df2.columns.drop(["jurisdiccion","departamento","sector","ambito","Nro_Encuestados"]).tolist()
df2[columnas_numericas] = df2[columnas_numericas].div(df2["Nro_Encuestados"], axis=0)

In [68]:
df1.head()

,NSE_Quintiles_Q1,NSE_Quintiles_Q2,NSE_Quintiles_Q3,NSE_Quintiles_Q4,NSE_Quintiles_Q5,Nivel_Ed_Madre_Blanco,Nivel_Ed_Madre_Multimarca,Nivel_Ed_Madre_No_disponible,Nivel_Ed_Madre_No_fue_a_la_escuela,Nivel_Ed_Madre_Primaria_completo,...,repitencia_Repitió_2_veces_o_más,repitencia__Blanco,sector,sobreedad_1_año_de_sobreedad_18_años_al_30_de_junio,sobreedad_2_años_de_sobreedad_19_años_al_30_de_junio,sobreedad_3_años_o_más_de_sobreedad_20_años_mas_30Jun,sobreedad_Blanco,sobreedad_Edad_teórica_para_el_año_17_años_al_30_de_junio,sobreedad_Menores_de_17_años,Nro_Encuestados
0,0.152405,0.197490,0.223798,0.202918,0.223389,0.0,0.0,0.0,0.007193,0.054917,...,0.041884,0.0,Estatal,0.176762,0.073805,0.037161,0.0,0.704795,0.007478,2004.231349
1,0.067775,0.105373,0.185564,0.256549,0.384740,0.0,0.0,0.0,0.004638,0.028583,...,0.017440,0.0,Privado,0.096645,0.015842,0.006071,0.0,0.872215,0.009229,1340.471615
2,0.149900,0.157649,0.232594,0.253117,0.206739,0.0,0.0,0.0,0.006213,0.051418,...,0.030347,0.0,Estatal,0.193596,0.064863,0.013900,0.0,0.716435,0.011207,1037.452244
3,0.032863,0.073797,0.153025,0.272731,0.467585,0.0,0.0,0.0,0.001008,0.017874,...,0.009748,0.0,Privado,0.075119,0.011127,0.008190,0.0,0.902621,0.002943,1318.705478
4,0.166372,0.186578,0.197982,0.281982,0.167086,0.0,0.0,0.0,0.009024,0.058463,...,0.021712,0.0,Estatal,0.181983,0.043063,0.039403,0.0,0.733501,0.002049,814.644791


In [69]:
df2.head()

,NSE_Quintiles_Q1,NSE_Quintiles_Q2,NSE_Quintiles_Q3,NSE_Quintiles_Q4,NSE_Quintiles_Q5,Nivel_Ed_Madre_Blanco,Nivel_Ed_Madre_Multimarca,Nivel_Ed_Madre_No_disponible,Nivel_Ed_Madre_No_fue_a_la_escuela,Nivel_Ed_Madre_Primaria_completo,...,repitencia_Repitió_2_veces_o_más,repitencia__Blanco,sector,sobreedad_1_año_de_sobreedad_18_años_al_30_de_junio,sobreedad_2_años_de_sobreedad_19_años_al_30_de_junio,sobreedad_3_años_o_más_de_sobreedad_20_años_mas_30Jun,sobreedad_Blanco,sobreedad_Edad_teórica_para_el_año_17_años_al_30_de_junio,sobreedad_Menores_de_17_años,Nro_Encuestados
0,0.155872,0.198645,0.217231,0.206032,0.222221,0.0,0.0,0.0,0.007152,0.055994,...,0.041597,0.0,Estatal,0.173921,0.075148,0.036295,0.0,0.707674,0.006962,2001.088457
1,0.066834,0.104470,0.184759,0.261522,0.382414,0.0,0.0,0.0,0.004762,0.028661,...,0.016206,0.0,Privado,0.098121,0.016216,0.006198,0.0,0.870110,0.009355,1339.742741
2,0.143804,0.152599,0.239605,0.257262,0.206731,0.0,0.0,0.0,0.004976,0.045573,...,0.027931,0.0,Estatal,0.197110,0.066703,0.014396,0.0,0.709792,0.012000,1041.247691
3,0.032941,0.071253,0.159066,0.271205,0.465536,0.0,0.0,0.0,0.000971,0.017017,...,0.009809,0.0,Privado,0.070326,0.011309,0.008225,0.0,0.905507,0.004634,1317.988433
4,0.171103,0.187624,0.197425,0.279435,0.164414,0.0,0.0,0.0,0.009161,0.054881,...,0.019302,0.0,Estatal,0.166892,0.045432,0.038140,0.0,0.745754,0.003783,797.550655


# Al pasarlo a frecuencias vamos a ver si hay diferencias

In [70]:
# Columnas comunes
columnas_comunes = df1.columns.intersection(df2.columns)

# Excluir columnas de tipo object (strings)
columnas_numericas = [col for col in columnas_comunes if pd.api.types.is_numeric_dtype(df1[col])]

# Seleccionamos solo columnas numéricas y hacemos copia
df1_comun = df1[columnas_numericas].copy()
df2_comun = df2[columnas_numericas].copy()

# Convertir números con coma decimal a float
for col in columnas_numericas:
    df1_comun[col] = df1_comun[col].astype(str).str.replace(',', '.').astype(float)
    df2_comun[col] = df2_comun[col].astype(str).str.replace(',', '.').astype(float)

# Restar DataFrames
diferencias = df1_comun - df2_comun

# Estadísticas
pd.set_option('display.max_rows', None)
#print(diferencias.describe())
print("Máxima diferencia por columna:\n", diferencias.max())
print("Máxima diferencia absoluta total:", diferencias.abs().max().max())

Máxima diferencia por columna:
 NSE_Quintiles_Q1                                                     0.287879
NSE_Quintiles_Q2                                                     0.066331
NSE_Quintiles_Q3                                                     0.066667
NSE_Quintiles_Q4                                                     0.070513
NSE_Quintiles_Q5                                                     0.090909
Nivel_Ed_Madre_Blanco                                                0.000000
Nivel_Ed_Madre_Multimarca                                            0.000000
Nivel_Ed_Madre_No_disponible                                         0.000000
Nivel_Ed_Madre_No_fue_a_la_escuela                                   0.047808
Nivel_Ed_Madre_Primaria_completo                                     0.156250
Nivel_Ed_Madre_Primaria_incompleto                                   0.065076
Nivel_Ed_Madre_Secundaria_completo                                   0.089167
Nivel_Ed_Madre_Secundaria_incomp

* la columna ap22c_Entre_5_y_10_horas_en_la_semana varia un 36% entre ambos data sets




sabiendo que en ambos datasets hay datos disertantes debido a un proceso de 
proteccion de datos personales, vamos a utilizar el promedio para unificar un dataset

In [71]:
# Columnas a excluir de df1
excluir = ["jurisdiccion","departamento","sector","ambito"]

# Columnas comunes (sin las excluidas)
comunes = [c for c in df1.columns.intersection(df2.columns) if c not in excluir]

# Promedio de las comunes
promedio = (df1[comunes] + df2[comunes]) / 2

# Columnas exclusivas de df2
solo_df2 = df2.drop(columns=comunes, errors="ignore")

# Unificación final: comunes promediadas + exclusivas de df2
resultado = pd.concat([promedio, solo_df2], axis=1)

In [72]:
resultado.head()

,NSE_Quintiles_Q1,NSE_Quintiles_Q2,NSE_Quintiles_Q3,NSE_Quintiles_Q4,NSE_Quintiles_Q5,Nivel_Ed_Madre_Blanco,Nivel_Ed_Madre_Multimarca,Nivel_Ed_Madre_No_disponible,Nivel_Ed_Madre_No_fue_a_la_escuela,Nivel_Ed_Madre_Primaria_completo,...,sobreedad_2_años_de_sobreedad_19_años_al_30_de_junio,sobreedad_3_años_o_más_de_sobreedad_20_años_mas_30Jun,sobreedad_Blanco,sobreedad_Edad_teórica_para_el_año_17_años_al_30_de_junio,sobreedad_Menores_de_17_años,Nro_Encuestados,ambito,departamento,jurisdiccion,sector
0,0.154138,0.198068,0.220514,0.204475,0.222805,0.0,0.0,0.0,0.007172,0.055456,...,0.074476,0.036728,0.0,0.706235,0.007220,2002.659903,Urbano,Comuna 1,Ciudad Autónoma de Buenos Aires,Estatal
1,0.067305,0.104922,0.185161,0.259035,0.383577,0.0,0.0,0.0,0.004700,0.028622,...,0.016029,0.006134,0.0,0.871162,0.009292,1340.107178,Urbano,Comuna 1,Ciudad Autónoma de Buenos Aires,Privado
2,0.146852,0.155124,0.236100,0.255190,0.206735,0.0,0.0,0.0,0.005595,0.048496,...,0.065783,0.014148,0.0,0.713113,0.011604,1039.349967,Urbano,Comuna 10,Ciudad Autónoma de Buenos Aires,Estatal
3,0.032902,0.072525,0.156045,0.271968,0.466560,0.0,0.0,0.0,0.000989,0.017445,...,0.011218,0.008207,0.0,0.904064,0.003789,1318.346955,Urbano,Comuna 10,Ciudad Autónoma de Buenos Aires,Privado
4,0.168737,0.187101,0.197703,0.280708,0.165750,0.0,0.0,0.0,0.009093,0.056672,...,0.044247,0.038772,0.0,0.739628,0.002916,806.097723,Urbano,Comuna 11,Ciudad Autónoma de Buenos Aires,Estatal


In [73]:
guardar_df(resultado, ruta_archivo_final)

Archivo guardado exitosamente en: ../data/processed/2024_Ap_56listo.csv


# Listo

Ya tenemos un dataset limpio para el año 2024.
La idea de este notebook es utilizarlo para todos los años restantes y hacer una union entre todos